In [ ]:
"""
To package Apical Filter into a function that
can be used to filter inhibitory/excitatory



"""

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from os import sys
sys.path.append("/meshAfterParty/")

In [4]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

import numpy as np

import proofreading_utils as pru

import preprocessing_vp2 as pre

# -- For the axon classification --

import neuron_searching as ns

import skeleton_utils as sk

import numpy_utils as nu

import networkx_utils as xu

import system_utils as su

INFO - 2021-01-22 09:49:50,188 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-22 09:49:50,189 - settings - Setting database.user to celiib
INFO - 2021-01-22 09:49:50,190 - settings - Setting database.password to newceliipass
INFO - 2021-01-22 09:49:50,202 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-01-22 09:49:50,203 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-22 09:49:50,218 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-01-22 09:49:50,489 - settings - Setting enable_python_native_blobs to True


In [5]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-22 09:50:01,725 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-22 09:50:02,051 - settings - Setting enable_python_native_blobs to True


# Importing the Neuron

In [6]:
neuron_file = "./864691135644605167_0_25"
neuron_obj = nru.decompress_neuron(neuron_file,neuron_file)

Decompressing Neuron in minimal output mode...please wait


In [7]:
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton


# The function that will return the apical branches on a certain limb

In [9]:
import networkx_utils as xu
import skeleton_utils as sk
import networkx as nx
import numpy_utils as nu

In [15]:
def apical_branch_candidates_on_limb(limb_obj,
                                     
                                    apical_check_distance_max = 100000,
                                    apical_check_distance_min = 25000,
                                    plot_restricted_skeleton = False,
                                    plot_restricted_skeleton_with_endnodes=False,
                                     
                                     
                                    angle_threshold = 30,
                                    top_volume_vector = np.array([0,-1,0]),
                                     
                                     spine_density_threshold = 0.00001,
                                    total_skeleton_distance_threshold_multiplier = 0.5,
                                    apical_width_threshold = 350,
                                    upward_distance_to_skeletal_distance_ratio_threshold = 0.85,
                                    
                                    verbose=False):
    """
    Purpose: To identify the branches on the limb that are most likely 
    part of a large upward apical branch
    
    
    Psuedoode:
    0a) Getting the subskeleton region to analyze
    0b) Divided the Restricted Skeleton into components to analyze
    
    For each connected component
    1) Get all the end nodes of the subgraph
    2) Subtract of the closest subgraph node to limb start
    For each end node
    3) Look at the vector between end nodes and closest node 
        (continue if not approximately straight up) and not long enough
    4) Find the branches that contain the two ends of the path

    For all combinations of branches:

    5) Find the shortest path between the two branches on the context network
    6) Get the subskeleton:
    - Analyze for width and spine density (and if too thin or not spiny enough then continue)
    7) If passed all tests then add the branch path as possible candidate
    
    """
    
    
    
    curr_limb = limb_obj
    apical_branches = []
    
    total_skeleton_distance_threshold = total_skeleton_distance_threshold_multiplier*(apical_check_distance_max - apical_check_distance_min)
    
    
    #0a) Getting the subskeleton region to analyze
    
    
    limb_gr = sk.convert_skeleton_to_graph(curr_limb.skeleton)
    st_node = xu.get_graph_node_by_coordinate(limb_gr,curr_limb.current_starting_coordinate)
    nodes_max_distance = xu.find_nodes_within_certain_distance_of_target_node(limb_gr,st_node,apical_check_distance_max)
    nodes_min_distance = xu.find_nodes_within_certain_distance_of_target_node(limb_gr,st_node,apical_check_distance_min)
    nodes_with_distance_range = np.setdiff1d(list(nodes_max_distance),list(nodes_min_distance))


    restricted_limb_gr = limb_gr.subgraph(nodes_with_distance_range)
    restricted_limb_sk = sk.convert_graph_to_skeleton(restricted_limb_gr)
    
    if plot_restricted_skeleton:
        nviz.plot_objects(curr_limb.mesh,
                         skeletons=[restricted_limb_sk])
        
        
    #0b) Divided the Restricted Skeleton into components to analyze
        
    conn_comp = list([np.array(list(k)) for k in nx.connected_components(restricted_limb_gr)])
    conn_comp_closest_nodes = [xu.shortest_path_between_two_sets_of_nodes(limb_gr,[st_node],k)[2]
                               for k in conn_comp]
    
    if plot_restricted_skeleton_with_endnodes:
        nviz.plot_objects(curr_limb.mesh,
                     skeletons=[restricted_limb_sk],
                     scatters=[xu.get_coordinate_by_graph_node(limb_gr,conn_comp_closest_nodes)],
                     scatter_size=1)
        
        
    
    for component_idx in range(len(conn_comp)):
        
        # 1) Get all the end nodes of the subgraph
        curr_cmpnt = conn_comp[component_idx]
        closest_node = conn_comp_closest_nodes[component_idx]
        closest_node_coordinate = xu.get_coordinate_by_graph_node(limb_gr,closest_node)

        c_subgraph = restricted_limb_gr.subgraph(curr_cmpnt)
        endnodes = xu.get_nodes_of_degree_k(c_subgraph,1)

        #2) Subtract of the closest subgraph node to limb start
        filtered_endnodes = np.setdiff1d(endnodes,closest_node)
        filtered_endnodes_coordinates = xu.get_coordinate_by_graph_node(limb_gr,filtered_endnodes)

        if verbose:
            print(f"Filered End nodes for component {component_idx}: {filtered_endnodes}")
            
        
        
        for e_node_idx in range(len(filtered_endnodes)):
            
            #3) Look at the vector between end nodes and closest node 
            e_node = filtered_endnodes[e_node_idx]
            e_node_coordinate = filtered_endnodes_coordinates[e_node_idx]

            # nviz.plot_objects(curr_limb.mesh,
            #                  skeletons=[restricted_limb_sk],
            #                  scatters=[xu.get_coordinate_by_graph_node(limb_gr,[closest_node,e_node])],
            #                  scatter_size=1)

            curr_vector = e_node_coordinate-closest_node_coordinate
            curr_vector_upward_distance = -curr_vector[1]
            curr_vector_len = np.linalg.norm(curr_vector)

            curr_vector_angle = nu.angle_between_vectors(top_volume_vector,curr_vector)

            if verbose:
                print(f"End Node Candidate {e_node_idx} angle = {np.round(curr_vector_angle,2)}"
                      f"\n    Upward distance {np.round(curr_vector_upward_distance,2)}")

            reject_flag = False
            if curr_vector_angle > angle_threshold:
                if verbose:
                    print(f"Rejecting candidate because did not pass angle threshold of ess than {angle_threshold}")
                continue
                
                
                
            #4) Find the branches that contain the two ends of the path
            curr_skeleton_path = sk.convert_graph_to_skeleton(limb_gr.subgraph(nx.shortest_path(limb_gr,closest_node,e_node)))
            curr_skeleton_path_len = sk.calculate_skeleton_distance(curr_skeleton_path)

            e_node_branches = nru.find_branch_with_specific_coordinate(curr_limb,e_node_coordinate)

            closest_node_branches =  nru.find_branch_with_specific_coordinate(curr_limb,closest_node_coordinate)

            #get all possible combinations
            all_branch_pairings = nu.unique_pairings_between_2_arrays(closest_node_branches,
                                                                      e_node_branches
                                                                     )
            if verbose:
                print(f"all_branch_pairings = {all_branch_pairings}")
                
                
                
            
            #for st_branch,end_branch in all_branch_pairings
            #5) Find the shortest path between the two branches on the context network

            for curr_pairing_idx  in range(len(all_branch_pairings)):

                st_branch = all_branch_pairings[curr_pairing_idx][0]
                end_branch = all_branch_pairings[curr_pairing_idx][1]

                try:
                    branch_path = nx.shortest_path(curr_limb.concept_network,st_branch,end_branch)
                except:
                    print(f"Couln't find path between branches")

                #6) Get the subskeleton:
                #- Analyze for width and spine density (and if too thin or not spiny enough then continue)

                #total_skeleton = sk.stack_skeletons([curr_limb[k].skeleton for k in branch_path])
                skeleton_distance_per_branch = np.array([sk.calculate_skeleton_distance(curr_limb[k].skeleton) for k in branch_path])
                branch_widths = np.array([curr_limb[k].width_new["median_mesh_center"] for k in branch_path])
                branch_spines = np.array([curr_limb[k].n_spines for k in branch_path])

                total_skeleton_distance = np.sum(skeleton_distance_per_branch)
                total_spine_density = np.sum(branch_spines)/np.sum(skeleton_distance_per_branch)
                scaled_branch_width = np.sum(skeleton_distance_per_branch*branch_widths)/(total_skeleton_distance)
                curr_vector_upward_distance
                upward_to_skeletal_length_ratio = curr_vector_upward_distance/curr_skeleton_path_len


                if verbose:
                    print(f"total_spine_density = {total_spine_density}")
                    print(f"scaled_branch_width = {scaled_branch_width}")
                    print(f"curr_skeleton_path_len = {curr_skeleton_path_len}")
                    print(f"curr_vector_upward_distance = {curr_vector_upward_distance}")
                    print(f"upward ratio to length = {upward_to_skeletal_length_ratio}")
                    
                # Apply the restrictions
                if ((total_spine_density > spine_density_threshold) and
                    (total_skeleton_distance > total_skeleton_distance_threshold) and 
                    (scaled_branch_width > apical_width_threshold) and
                    (upward_to_skeletal_length_ratio > upward_distance_to_skeletal_distance_ratio_threshold)):
                    
                    print(f"Adding the following branch path as a apical pathway: {branch_path}")
                    apical_branches += list(branch_path)
                else:
                    print("Did not pass final filters to continuing")
                    continue
    
    return np.unique(apical_branches)
            


# Larger Funtion that will find apical candidates for all limbs 

In [20]:
import proofreading_utils as pru

In [27]:
def apical_classification(neuron_obj,
                          apical_soma_angle_threshold=40,
                          plot_viable_limbs = False,
                          label_neuron_branches=True,
                          plot_apical_branches=True,
                          verbose=False,
                         **kwargs):
    """
    Will compute a limb branch dict of all 
    the branches that are part of a probably 
    long reaching apical branch
    
    Pseudocode: 
    1) Split the neuron and take the first neuron obj (assume only some in neuron)
    2) Check only 1 soma 
    3) Filter the limbs for viable aplical limbs based on the soma angle
    4) Iterate through the viable limbs to find the apical branches on each limb
    
    Ex:
    apical_classification(neuron_obj,
                          apical_soma_angle_threshold=40,
                          plot_viable_limbs = False,
                          label_neuron_branches=True,
                          plot_apical_branches=True,
                          verbose=False)
    """
    
    split_time = time.time()
    neuron_obj_list = pru.split_neuron(neuron_obj,
                                      plot_seperated_neurons=False,
                                      verbose=verbose)
    
    if verbose:
        print(f"Total time for split = {time.time() - split_time}")
        
    if len(neuron_obj_list)==0:
        raise Exception(f"Split Neurons not just one: {neuron_obj_list}")
        
    
    curr_neuron_obj = neuron_obj
    
    
    viable_limbs = nru.viable_axon_limbs_by_starting_angle(curr_neuron_obj,
                                       soma_angle_threshold=apical_soma_angle_threshold,
                                       above_threshold=False,
                                       verbose=verbose)

    if verbose:
        print(f"viable_limbs = {viable_limbs}")
        
        
    if plot_viable_limbs:
        ret_col = nviz.visualize_neuron(curr_neuron_obj,
                     visualize_type=["mesh","skeleton"],
                     limb_branch_dict={f"L{k}":"all" for k in viable_limbs},
                     return_color_dict=True)
        
    
    apical_limb_branch_dict = dict()
    
    for limb_idx in viable_limbs:
        
        curr_limb = curr_neuron_obj[limb_idx]

        if verbose:
            print(f"Working on limb {limb_idx}")
        
        curr_limb_apical_branches = apical_branch_candidates_on_limb(curr_limb,
                                         verbose=verbose,
                                         **kwargs)
        
        apical_limb_branch_dict.update({f"L{limb_idx}":curr_limb_apical_branches})
        
    
    if plot_apical_branches:
        nviz.visualize_neuron(curr_neuron_obj,
                             visualize_type=["mesh"],
                             limb_branch_dict=apical_limb_branch_dict,
                             mesh_color="blue",
                             mesh_whole_neuron=True,
                             mesh_color_alpha=1)
        
    if label_neuron_branches:
        nru.add_branch_label(curr_neuron_obj,
                    limb_branch_dict=apical_limb_branch_dict,
                    labels="apical")
        
    return apical_limb_branch_dict

In [30]:
import classification_utils as clu

In [32]:
clu.apical_classification(neuron_obj,
                          apical_soma_angle_threshold=40,
                          plot_viable_limbs = False,
                          label_neuron_branches=True,
                          plot_apical_branches=True,
                          verbose=False)



------ Working on Soma 0 -------
local_floating_meshes = []
Processing Neuorn in minimal output mode...please wait
Total time for neuron instance creation = 2.052189826965332


Number of seperate neuron objects = 1
functions_list = [<function soma_starting_angle at 0x7ff4200272f0>]
Adding the following branch path as a apical pathway: [4, 5, 7]

 Working on visualization type: mesh


{'L0': array([4, 5, 7])}